In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-12 04:58:41--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.25MB/s    in 0.2s    

2022-09-12 04:58:41 (6.25 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 http:

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes','total_votes','vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26MV8D0KG6QI6|          5|            0|          0|   N|                Y|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|                Y|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|                N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|                Y|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|                Y|
| RIG9AWFOGRDVO|          2|            1|          1|   N|                Y|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|                Y|
| RB15NBVY5ELVW|          5|            2|          2|   N|                Y|
| R56358YM1ZJ7I|          5|            0|          0|   N|                N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|     

In [ ]:
# Filter total votes cout is equal to ot greater than 20
total_votes_df = vine_df.filter("total_votes>=20")
total_votes_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
|R33VIWRD2X5IA9|          1|            0|         20|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R26UFXLLQ2N0CR|          1|            0|         20|   N|                Y|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|     

In [ ]:
#filter helpful_votes / total_votes is equeal to or greater than 50%
helpful_votes_df = total_votes_df.filter((total_votes_df["helpful_votes"] /total_votes_df["total_votes"]) >= .5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|                N|
| RO6JNMUJDE1LP|          5|           37|         50|   N|                N|
|R1WNL4OIMSKDBR|          5|           35|         36|   N|     

In [ ]:
# Retrieves all rows where a review was written as part of the Vine program
vine_reviews_df = helpful_votes_df.filter(helpful_votes_df["vine"] == 'Y')
vine_reviews_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3MKO875WGIEBJ|          5|           34|         37|   Y|                N|
|R3OPNTK61FQ7MP|          5|           32|         37|   Y|                N|
| RMDRU8I773X5U|          4|           22|         25|   Y|                N|
| RR83RGWFEFZCL|          3|           61|         64|   Y|                N|
|R2TMZ7GV8SPIRV|          4|           23|         23|   Y|                N|
| RX88JLUIFA4Q2|          4|           84|         87|   Y|                Y|
| RV3AHTDRHDH7R|          5|          253|        258|   Y|                N|
|R13EGJI8C8YJ4Y|          5|           19|         20|   Y|                N|
| R9SNSQTK4TJOG|          2|           11|         20|   Y|                N|
|R1A76AK493C8P8|          5|           83|         90|   Y|     

In [ ]:
# Retrieves all rows where a review was written as part of the Vine program
not_vine_reviews_df = helpful_votes_df.filter(helpful_votes_df["vine"] == 'N')
not_vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|                N|
| RO6JNMUJDE1LP|          5|           37|         50|   N|                N|
|R1WNL4OIMSKDBR|          5|           35|         36|   N|     

In [ ]:
# Determine total number of reviews (PAID)
paid_reviews = vine_reviews_df.count()
paid_reviews

61

In [ ]:
# Determine number of 5-star reviews (PAID)
paid_five_star_reviews = vine_reviews_df.filter(vine_reviews_df["star_rating"] == '5').count()
paid_five_star_reviews

20

In [ ]:
# Determine percentage of 5 start reviews (PAID)
paid_reviews_percentage = paid_five_star_reviews / paid_reviews
paid_reviews_percentage

0.32786885245901637

In [ ]:
# Determine total number of reviews (UNPAID)
unpaid_reviews = not_vine_reviews_df.count()
unpaid_reviews

28287

In [ ]:
# Determine number of 5-star reviews (UNPAID)
unpaid_five_star_reviews = not_vine_reviews_df.filter(not_vine_reviews_df["star_rating"] == '5').count()
unpaid_five_star_reviews

15689

In [ ]:
# Determine percentage of 5 start reviews (UNPAID)
unpaid_reviews_percentage = unpaid_five_star_reviews / unpaid_reviews
unpaid_reviews_percentage

0.5546364054159154

In [ ]:
# Retrieves all rows where a review was not written as part of the Vine program and purchase was verified
not_vine_reviews_purchased = helpful_votes_df.filter((helpful_votes_df["vine"] == 'N') & (helpful_votes_df["verified_purchase"] == 'Y'))
not_vine_reviews_purchased.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R1WNL4OIMSKDBR|          5|           35|         36|   N|                Y|
| RDGWZ3U8LQZE6|          2|           48|         54|   N|                Y|
| R22M0N2OZSICN|          5|           21|         23|   N|                Y|
|R1E43139TRHB5T|          5|           22|         22|   N|     

In [ ]:
# Determine total number of reviews + purchased (UNPAID)
unpaid_purchased_reviews = not_vine_reviews_purchased.count()
unpaid_purchased_reviews

17347

In [ ]:
# Determine number of 5-star reviews + purchased (UNPAID)
unpaid_purchased_five_star_reviews = not_vine_reviews_purchased.filter(not_vine_reviews_df["star_rating"] == '5').count()
unpaid_purchased_five_star_reviews

9858

In [ ]:
# Determine percentage of 5 start reviews + purchased (UNPAID)
unpaid_purchased_reviews_percentage = unpaid_purchased_five_star_reviews / unpaid_purchased_reviews
unpaid_purchased_reviews_percentage

0.5682827001787053